# Preparation

## Authentication

Before requests to orchestration can be issued, we need to provide authentication details to the SDK. This can be done either via a configuration file or via the environment. Make sure to check out the [SAP generative AI hub SDK project description](https://pypi.org/project/generative-ai-hub-sdk/) for more details. Below you will find an example for authenticating via environment variables using this very notebook.

In [ ]:
import os

os.environ["AICORE_AUTH_URL"] = "redacted"
os.environ["AICORE_BASE_URL"] = "redacted"
os.environ["AICORE_CLIENT_ID"] = "redacted"
os.environ["AICORE_CLIENT_SECRET"] = "redacted"
os.environ["AICORE_ORCHESTRATION_DEPLOYMENT_URL"] = "redacted"
os.environ["AICORE_RESOURCE_GROUP"] = "redacted"

You will need to authenticate in every exercise.

## Intializing the Orchestration Service

Typically, a virtual deployment of Orchestration must be configured before any interactions can occur. Once deployed, you will have access to a unique endpoint URL. For this session, the deployment has already been created for you. The corresponding URL, along with the authentication credentials, have been provided to you in advance. If you've adhered to the prerequisite guidelines, the URL should already be present in the environment variable `AICORE_ORCHESTRATION_DEPLOYMENT_URL`.

# Templating

Now that everything is prepared, we can write our first basic orchestration pipeline. The first fundamental module we will look at is templating. The templating module provides capabilities to define prompt skeletons that can then be parameterized per inference call. To check out how this works we first up select a Large Language Model (LLM) that will be used for inference.

In [ ]:
from gen_ai_hub.orchestration.models.llm import LLM

llm = LLM(
    name="gemini-1.5-flash",
    version="latest",
    parameters={"max_tokens": 256, "temperature": 0.2},
)

Now we can create a template using the template object provided by the SAP generative AI hub SDK.

In [ ]:
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue

template = Template(
    messages=[
        SystemMessage("You are a helpful translation assistant."),
        UserMessage(
            "Translate the following text to {{?to_lang}}: {{?text}}",
        ),
    ],
    defaults=[
        TemplateValue(name="to_lang", value="English"),
    ],
)

The code above creates a template that provides

- a system message,
- a user message that leverages templating syntax,
- default values for the introduced template parameters.

Currently there are three message types available:
- `SystemMessage`: A message for priming AI behavior. The system message is usually passed in as the first of a sequence of input messages.
- `UserMessage`: A message from a user.
- `AssistantMessage`: A message of the LLM.

Parameters are defined within the message string using the following syntax: `{{?param_name}}`.

Next up we create a orchestration configuration from the created objects.

In [ ]:
from gen_ai_hub.orchestration.models.config import OrchestrationConfig

config = OrchestrationConfig(
    template=template,
    llm=llm,
)

Lastly, we can call the orchestration service. Note that the actual template values are now passed to the `run` method. The `TemplateValue` name parameter corresponds to the parameter name `text` provided in the user message string. The parameter `to_lang` is omitted and the default defined in the PromptTemplate is used.

In [ ]:
from gen_ai_hub.orchestration.service import OrchestrationService

orchestration_service = OrchestrationService(
    api_url=os.environ["AICORE_ORCHESTRATION_DEPLOYMENT_URL"],
    config=config,
)
result = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="text",
            value="Interaktives Lernen mit SAP.",
        )
    ]
)
print(result.orchestration_result.choices[0].message.content)

# Model Harmonization

Orchestration harmonizes model usage, removing the need for prompting each model in vendor specific fashion. You can easily switch between a variety of models. Check out [this SAP Note](https://me.sap.com/notes/3437766) for further information regarding model availability. For this Jump-Start session model access is restricted to `gemini-1.5-flash` and `meta--llama3-70b-instruct` though. The code below will demonstrate how to easily switch between models, based on the templating code above!

In [ ]:
orchestration_service.config.llm = LLM(
    name="meta--llama3-70b-instruct",
)
result = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="text",
            value="Interaktives Lernen mit SAP.",
        )
    ]
)
print(result.orchestration_result.choices[0].message.content)

You can switch between those two models and compare their responses throughout all exercises. Simply change the `name` parameter of the LLM module configuration.

# Summary

Within this exercise you learned how to create a basic Orchestration pipeline that uses the Templating module. Also, you changed the model used for inference with ease. Let's explore more modules in the following exercises. Continue to [Exercise 2 - Orchestration Content Filtering](./ex2.ipynb).